In [3]:
pip install lxml Pyarrow requests pandas


  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB 330.3 kB/s eta 0:00:35
   ---------------------------------------- 0.0/11.5 MB 330.3 kB/s eta 0:00:35
   ---------------------------------------- 0.1/11.5 MB 655.4 kB/s eta 0:00:18
    --------------------------------------- 0.2/11.5 MB 958.1 kB/s eta 0:00:12
    --------------------------------------- 0.3/11.5 MB 1.3 MB/s eta 0:00:09
   - -------------------------------------- 0.3/11.5 MB 1.4 MB/s eta 0:00:09
   - -------------------------------------- 0.4/11.5 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.5/11.5 MB 1.5 MB/s eta 0:00:08
   -- ------------------------------------- 0.7/11.5 MB 1.7 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/11.5 MB 1.9 MB/s eta 0:00:06
   --- ------------

In [3]:
import pandas as pd
import requests 
from datetime import datetime
from io import StringIO


In [4]:
# Parameters for URL

report_type = 'HistoricalPoolPriceReportServlet' # Type of report
start_date = '02282020' # Start date for the data range of the report: February 28, 2020
end_date = '02282021' # End date for the data range of the report: February 28, 2021
content_type = 'html' # Type of content expected from the AESO server

# Date range check "Manually"

start_date_datatime = datetime.strptime(start_date, '%m%d%Y')
end_date_datatime = datetime.strptime(end_date, '%m%d%Y')
time_delta = end_date_datatime - start_date_datatime
time_delta

# Date range check "Conditional statement"

if time_delta.days <= 366:
    print("The selected data range is within the acceptable limit of 366 days.")
elif time_delta.days >= 366:
    print("Please select a new data range as this report is available for a maximum of 366 days of data.")
    import sys
    sys.exit(1)

# URL for web scrape
url = 'http://ets.aeso.ca/ets_web/ip/Market/Reports/{}?beginDate={}&endDate={}&contentType={}'.format(report_type, start_date, end_date, content_type)
print(url)

''' The placeholders {} in the URL string will be replaced with the values of report_type, 
start_date, end_date, and content_type
'''

# Get HTML content using the requests library
source = requests.get(url).text  # The .text attribute extracts the HTML content as plain text

# Wrap HTML content in StringIO object
html_buffer = StringIO(source)

# Parse the HTML content (stored in source) into a list of DataFrames 
df_list = pd.read_html(html_buffer)  # pd.read_html() searches for table elements and creates a df for each of them
print('number of tables =',  len(df_list)) 

# Create a DataFrame containing the data stored on table #2 (Index = 1) in the HTML
df = df_list[1]

# Display the DataFrame
df

The selected data range is within the acceptable limit of 366 days.
http://ets.aeso.ca/ets_web/ip/Market/Reports/HistoricalPoolPriceReportServlet?beginDate=02282020&endDate=02282021&contentType=html
number of tables = 2


,Date (HE),Price ($),30Ravg ($),AIL Demand (MW)
0,02/28/2020 01,29.31,36.10,9358.0
1,02/28/2020 02,29.89,36.10,9228.0
2,02/28/2020 03,29.94,36.09,9161.0
3,02/28/2020 04,29.88,36.08,9140.0
4,02/28/2020 05,29.84,36.08,9186.0
...,...,...,...,...
8779,02/27/2021 20,42.21,149.96,10729.0
8780,02/27/2021 21,26.74,149.92,10680.0
8781,02/27/2021 22,27.18,149.89,10567.0
8782,02/27/2021 23,35.90,149.88,10359.0
